In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

import torch
device = torch.device("cuda")
import torch.nn.functional as F

def test(model, test_loader):
    model.eval()
    correct = 0
    with torch.no_grad(): 
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output, *_ = model(data)
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            _, idx = output.max(dim=1)
            correct += (idx == target).sum().item()

    print('Test set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [2]:
from torch.utils import data
from torchvision import datasets
from torchvision import transforms

train_loader = data.DataLoader(
        datasets.MNIST('./data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=256, shuffle=True, drop_last=True)

test_loader = data.DataLoader(
        datasets.MNIST('./data', train=False,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=2048, shuffle=False, drop_last=False)

In [3]:
import torch.nn as nn
from torch.optim import Adam

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 32, 5, 1),
            nn.BatchNorm2d(32),
            nn.ReLU(True),
            nn.MaxPool2d(2)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(32, 64, 5, 1),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.MaxPool2d(2)
        )
        
        self.fc1 = nn.Sequential(
            nn.Linear(1024, 512),
            nn.ReLU(True)
        )
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        conv1 = self.conv1(x)
        conv2 = self.conv2(conv1)
        flatten = conv2.view(x.shape[0], -1)        
        fc1 = self.fc1(flatten)
        fc2 = self.fc2(fc1)
        return fc2, [conv1, conv2]

In [4]:
import numpy as np
from large_margin import LargeMarginLoss

lm = LargeMarginLoss(
    gamma=10000,
    alpha_factor=4,
    top_k=1,
    dist_norm=np.inf
)


def train_lm(model, train_loader, optimizer, epoch, lm):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data = data.to(device)
        one_hot = torch.zeros(len(target), 10).scatter_(1, target.unsqueeze(1), 1.).float()
        one_hot = one_hot.cuda()
        optimizer.zero_grad()
        output, feature_maps = model(data)
        #loss = F.mse_loss(output, target) * 5e-4 # l2_loss_weght
        loss = lm(output, one_hot, feature_maps)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

net = Net().to(device)
# net = nn.DataParallel(net).to(device)
optim = Adam(net.parameters())
for i in range(0, 5):
    train_lm(net, train_loader, optim, i, lm)
    test(net, test_loader)

Train Epoch: 0 [0/60000 (0%)]	Loss: 0.020817
Train Epoch: 0 [25600/60000 (43%)]	Loss: -2673.809082
Train Epoch: 0 [51200/60000 (85%)]	Loss: -4044.697266
Test set: Accuracy: 9753/10000 (98%)

Train Epoch: 1 [0/60000 (0%)]	Loss: -3921.711914
Train Epoch: 1 [25600/60000 (43%)]	Loss: -4475.788574
Train Epoch: 1 [51200/60000 (85%)]	Loss: -4430.020508
Test set: Accuracy: 9844/10000 (98%)

Train Epoch: 2 [0/60000 (0%)]	Loss: -4218.796387
Train Epoch: 2 [25600/60000 (43%)]	Loss: -4511.797852
Train Epoch: 2 [51200/60000 (85%)]	Loss: -4571.249023
Test set: Accuracy: 9850/10000 (98%)

Train Epoch: 3 [0/60000 (0%)]	Loss: -4648.967773
Train Epoch: 3 [25600/60000 (43%)]	Loss: -4678.980469
Train Epoch: 3 [51200/60000 (85%)]	Loss: -4658.126953
Test set: Accuracy: 9877/10000 (99%)

Train Epoch: 4 [0/60000 (0%)]	Loss: -4567.231445
Train Epoch: 4 [25600/60000 (43%)]	Loss: -4581.940430
Train Epoch: 4 [51200/60000 (85%)]	Loss: -4863.177734
Test set: Accuracy: 9888/10000 (99%)



In [6]:
def train_ce(model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output, _ = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

net = Net().to(device)
# net = nn.DataParallel(net).to(device)
optim = Adam(net.parameters())
for i in range(0, 5):    
    train_ce(net, train_loader, optim, i)
    test(net, test_loader)

Train Epoch: 0 [0/60000 (0%)]	Loss: 2.312209
Train Epoch: 0 [25600/60000 (43%)]	Loss: 0.057380
Train Epoch: 0 [51200/60000 (85%)]	Loss: 0.069225
Test set: Accuracy: 9871/10000 (99%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.021330
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.058321
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.032983
Test set: Accuracy: 9887/10000 (99%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.020935
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.051123
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.020498
Test set: Accuracy: 9911/10000 (99%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.018395
Train Epoch: 3 [25600/60000 (43%)]	Loss: 0.030107
Train Epoch: 3 [51200/60000 (85%)]	Loss: 0.018094
Test set: Accuracy: 9903/10000 (99%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.016279
Train Epoch: 4 [25600/60000 (43%)]	Loss: 0.013521
Train Epoch: 4 [51200/60000 (85%)]	Loss: 0.013503
Test set: Accuracy: 9915/10000 (99%)

